# 사용자 정의 층
https://www.tensorflow.org/tutorials/customization/custom_layers?hl=ko


In [2]:
import tensorflow as tf

print(tf.config.list_physical_devices('GPU'))


[]


In [19]:
# 층 : 유용한 연산자 집합

# 레이어 객체 : 파라미터1은 출력 차원(혹은 노드나 뉴런)이고 파라미터2은 출력 채널(예시로 RGB는 3)이다.

import tensorflow as tf

layer = tf.keras.layers.Dense(100)
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))
layer(tf.zeros([10, 5]))

print(layer.variables)  # 레이어의 학습 가능한 모든 변수들을 리스트 형태로 반환
print(f"레이어의 가중치 : {layer.kernel}")
print(f"레이어의 편향   : {layer.bias}")


[<KerasVariable shape=(5, 10), dtype=float32, path=dense_17/kernel>, <KerasVariable shape=(10,), dtype=float32, path=dense_17/bias>]
레이어의 가중치 : <KerasVariable shape=(5, 10), dtype=float32, path=dense_17/kernel>
레이어의 편향   : <KerasVariable shape=(10,), dtype=float32, path=dense_17/bias>


In [36]:
import tensorflow as tf

# 사용자 정의 층 구현

class MyDenseLayer(tf.keras.layers.Layer):

    def __init__(self, num_outputs):                  # 모든 입력 독립적 초기화 수행
        super(MyDenseLayer, self).__init__()          # 부모 클래스인 tf.keras.layers.Layer의 생성자를 호출해 레이어를 초기화
        self.num_outputs = num_outputs                # 레이어가 가질 출력 차원의 크기를 설정

    def build(self, input_shape):                     # 입력 텐서의 형상을 통해 나머지 초기화 작업 수행
        self.kernel = self.add_weight(                # .add_weight() : 레이어에 학습 가능한 가중치 kernel을 추가
            name="kernel",                            # 변수의 이름 지정
            shape=(input_shape[-1], self.num_outputs) # shape=(p1,p2) : p1은 입력 차원 수. p2는 출력 차원 수.
        )

    def call(self, inputs):                           # 순방향 계산 수행
        return tf.matmul(inputs, self.kernel)         # 행렬곱셈. inputs의 크기가 (batch_size, input_dim)이고 kernel의 크기가 (input_dim, num_outputs)라면, 결과는 (batch_size, num_outputs) 형태

# 클래스 사용
layer = MyDenseLayer(10)                                  # 출력 차원 수(뉴런)가 10개인 덴스 레이어 객체 생성
print(tf.zeros([10, 5]).shape)                            # 사용할 레이어의 형태 확인
_ = layer(tf.zeros([10, 5]))                              # 반환값을 _에 저장하고 해당 코드가 마지막에 있을 때는 반환값 미출력.10행 5열 형태의 텐서를 모든 원소를 0으로 초기화. 추후에 해당 레이어는 inputs (10,5) 크기와 kernel (5,10) 크기로 행렬 곱셉하기 때문에 결과는 (10,10)으로 나온다.
print([var.name for var in layer.trainable_variables])    # 학습 가능한 변수 출력


(10, 5)
['kernel']


In [46]:
import tensorflow as tf

# 모델: 층 구성

# 클래스 선언

class ResentIdentityBlock(tf.keras.Model):

  def __init__(self, kernel_size, filters):
    super(ResentIdentityBlock, self).__init__(name='')
    filters1, filters2, filters3 = filters
    self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
    self.bn2a = tf.keras.layers.BatchNormalization()
    self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
    self.bn2b = tf.keras.layers.BatchNormalization()
    self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
    self.bn2c = tf.keras.layers.BatchNormalization()

  def call(self, input_tensor, training=False):
    if len(input_tensor.shape) == 3: input_tensor = tf.expand_dims(input_tensor, axis=-1)  # 입력 텐서 차원 확인 후, 채널 차원 추가
    x = self.conv2a(input_tensor)
    x = self.bn2a(x, training=training)
    x = tf.nn.relu(x)
    x = self.conv2b(x)
    x = self.bn2b(x, training=training)
    x = tf.nn.relu(x)
    x = self.conv2c(x)
    x = self.bn2c(x, training=training)
    x += input_tensor
    return tf.nn.relu(x)

# 클래스 사용

block = ResentIdentityBlock(1, [1, 2, 3])
_ = block(tf.zeros([1, 2, 3, 3]))  # 입력 텐서 크기: (배치 크기 1, 높이 2, 너비 3, 채널 3)
print(block.layers)
print(block.variables)
print(len(block.variables))
block.summary()


[<Conv2D name=conv2d_24, built=True>, <BatchNormalization name=batch_normalization_22, built=True>, <Conv2D name=conv2d_25, built=True>, <BatchNormalization name=batch_normalization_23, built=True>, <Conv2D name=conv2d_26, built=True>, <BatchNormalization name=batch_normalization_24, built=True>]
[<KerasVariable shape=(1, 1, 3, 1), dtype=float32, path=conv2d_24/kernel>, <KerasVariable shape=(1,), dtype=float32, path=conv2d_24/bias>, <KerasVariable shape=(1,), dtype=float32, path=batch_normalization_22/gamma>, <KerasVariable shape=(1,), dtype=float32, path=batch_normalization_22/beta>, <KerasVariable shape=(1,), dtype=float32, path=batch_normalization_22/moving_mean>, <KerasVariable shape=(1,), dtype=float32, path=batch_normalization_22/moving_variance>, <KerasVariable shape=(1, 1, 1, 2), dtype=float32, path=conv2d_25/kernel>, <KerasVariable shape=(2,), dtype=float32, path=conv2d_25/bias>, <KerasVariable shape=(2,), dtype=float32, path=batch_normalization_23/gamma>, <KerasVariable shape

Model: ""

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_24 (Conv2D)                   │ (1, 2, 3, 1)                │               4 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_22               │ (1, 2, 3, 1)                │               4 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_25 (Conv2D)                   │ (1, 2, 3, 2)                │               4 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_23               │ (1, 2, 3, 2)                │               8 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_26 (Conv2D)                   │ (1, 2, 3, 3)                │               9 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_24               │ (1, 2, 3, 3)                │              12 │
│ (BatchNormalization)                 │                             │                 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 41 (164.00 B)

 Trainable params: 29 (116.00 B)

 Non-trainable params: 12 (48.00 B)

In [47]:
# 단순하게 층을 순서대로 하나씩 호출

import tensorflow as tf

my_seq = tf.keras.Sequential([
    tf.keras.layers.Conv2D(1, (1, 1), input_shape=(None, None, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(2, 1, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(3, (1, 1)),
    tf.keras.layers.BatchNormalization()
])

my_seq(tf.zeros([1, 2, 3, 3]))

my_seq.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_27 (Conv2D)                   │ (None, None, None, 1)       │               4 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_25               │ (None, None, None, 1)       │               4 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_28 (Conv2D)                   │ (None, None, None, 2)       │               4 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_26               │ (None, None, None, 2)       │               8 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_29 (Conv2D)                   │ (None, None, None, 3)       │               9 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_27               │ (None, None, None, 3)       │              12 │
│ (BatchNormalization)                 │                             │                 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 41 (164.00 B)

 Trainable params: 29 (116.00 B)

 Non-trainable params: 12 (48.00 B)